### MNIST multi GPUs implementation
inspiration from https://keras.io/guides/distributed_training_with_tensorflow/

In [13]:
import tensorflow as tf
from keras.datasets import mnist
#from tensorflow.keras.datasets import mnist
# from tensorflow import keras
# from tensorflow.keras import layers
import keras
from keras import layers

In [14]:
def get_dataset():
    # batch_size = 32
    
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    #print(x_test.shape)

    x_train = x_train.reshape((60000, 28 * 28))
    x_train = x_train.astype("float32") / 255
    x_test = x_test.reshape((10000, 28 * 28))
    x_test = x_test.astype("float32") / 255
    y_train = y_train.astype("float32")
    y_test = y_test.astype("float32")

    return (
        #tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size),
        #tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size),
        (x_train, y_train),
        (x_test, y_test)
    )

In [15]:
def get_compiled_model():
    # Make a simple 2-layer densely-connected neural network.
    model = keras.Sequential([
        layers.Dense(512, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])

    model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
    return model

In [18]:
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    model = get_compiled_model()

    # Train the model on all available devices.
    (x_train, y_train), (x_test, y_test) = get_dataset()
    model.fit(x_train, y_train, epochs=5, batch_size=64)

    # Test the model on all available devices.
    test_loss, test_acc = model.evaluate(x_test, y_test)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Epoch 1/5
INFO:tensorflow:Collective all_reduce tensors: 4 all_reduces, num_devices = 2, group_size = 2, implementation = CommunicationImplementation.NCCL, num_packs = 1
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8927 - loss: 0.3715
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9720 - loss: 0.0946
Epoch 3/5
 22/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9869 - loss: 0.0427    

2025-05-26 21:38:25.589343: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9820 - loss: 0.0619
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9873 - loss: 0.0413
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9913 - loss: 0.0302
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9784 - loss: 0.0763


In [17]:
print(f"test_acc: {test_acc}")

test_acc: 0.982200026512146
